# Marginal distance between clusters

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 200

import seaborn as sns
sns.set(rc={'figure.figsize':(15,5)})

import matplotlib.pyplot as plt
import numpy as np

In [9]:
merchant_agg = pd.read_csv("../data/curated/clusters/input/agg_transaction_train.csv")[['merchant_abn', 'total_number_of_distinct_customers', 
    'monthly_average_number_of_orders', 'monthly_average_bnpl_revenue']]
merchant_clusters = pd.read_csv("../data/curated/clusters/output/merchant_clusters.csv")

display(merchant_agg.head(3))
display(merchant_clusters.head(3))

,merchant_abn,total_number_of_distinct_customers,monthly_average_number_of_orders,monthly_average_bnpl_revenue
0,10023283211,2619,154.333333,33301.586223
1,10342410215,729,40.888889,15624.420327
2,10346855916,7,1.400000,2186.800497


,merchant_abn,label
0,10023283211,2
1,10342410215,2
2,10346855916,0


**Merge datasets**

In [10]:
merged_feature_cluster = merchant_clusters.merge(merchant_agg, on='merchant_abn', how='inner').drop('merchant_abn', axis=1)
display(merged_feature_cluster.head(3))

,label,total_number_of_distinct_customers,monthly_average_number_of_orders,monthly_average_bnpl_revenue
0,2,2619,154.333333,33301.586223
1,2,729,40.888889,15624.420327
2,0,7,1.400000,2186.800497


**Perfrom log transformation**

In [11]:
agg_byCluster = pd.DataFrame(np.log(merged_feature_cluster.iloc[:,1:])).add_prefix('log_')
agg_byCluster['cluster'] = pd.Categorical(merged_feature_cluster.label)

display(agg_byCluster.head(3))
display(agg_byCluster.groupby('cluster').describe().T)

,log_total_number_of_distinct_customers,log_monthly_average_number_of_orders,log_monthly_average_bnpl_revenue,cluster
0,7.870548,5.039115,10.413360,2
1,6.591674,3.710858,9.656590,2
2,1.945910,0.336472,7.690195,0


cluster                                                 0           1  \
log_total_number_of_distinct_customers count  2260.000000  433.000000   
                                       mean      4.401559    9.122465   
                                       std       1.425701    0.527432   
                                       min       0.000000    6.797940   
                                       25%       3.555348    8.798153   
                                       50%       4.620047    9.095378   
                                       75%       5.529429    9.477233   
                                       max       6.680855   10.089137   
log_monthly_average_number_of_orders   count  2260.000000  433.000000   
                                       mean      1.742218    6.570714   
                                       std       1.053807    0.822478   
                                       min       0.000000    3.932914   
                                       25%       0.823200    6.062492   
                                       50%       1.744357    6.432225   
                                       75%       2.646963    6.946762   
                                       max       3.796737    9.529497   
log_monthly_average_bnpl_revenue       count  2260.000000  433.000000   
                                       mean      7.888268   11.162768   
                                       std       0.835454    1.005125   
                                       min       5.122598    8.800315   
                                       25%       7.281548   10.596537   
                                       50%       7.725032   10.967890   
                                       75%       8.597238   11.939978   
                                       max       9.742062   13.056325   

cluster                                                 2  
log_total_number_of_distinct_customers count  1306.000000  
                                       mean      7.219092  
                                       std       0.863588  
                                       min       0.693147  
                                       25%       6.735185  
                                       50%       7.309878  
                                       75%       7.884952  
                                       max       8.580731  
log_monthly_average_number_of_orders   count  1306.000000  
                                       mean      4.371940  
                                       std       0.870887  
                                       min       0.693147  
                                       25%       3.861899  
                                       50%       4.447864  
                                       75%       5.052701  
                                       max       5.806807  
log_monthly_average_bnpl_revenue       count  1306.000000  
                                       mean      9.953495  
                                       std       0.994797  
                                       min       6.480134  
                                       25%       9.272875  
                                       50%      10.203986  
                                       75%      10.747229  
                                       max      11.795311

## Plot the Marginal Distances

In [12]:
def marginal_error(x_variable, agg_byCluster=agg_byCluster):
    '''this function creates plots of marginal distances between each cluster with respect to each feature'''
    sns.scatterplot(data=agg_byCluster, hue='cluster', style='cluster', x=x_variable, y=agg_byCluster['cluster'], palette='deep')

    # compute boundaries for clusters, outliers are removed using 1.5IQR
    description = agg_byCluster.groupby('cluster')[x_variable].describe().T
    description.loc["lb",:] = description.loc["25%"]-1.5*(description.loc["75%"]-description.loc["25%"])
    description.loc["ub",:] = description.loc["75%"]+1.5*(description.loc["75%"]-description.loc["25%"])
    description.loc["chosen_lower"] = np.where(description.loc["lb",:] < description.loc["min",:], description.loc["min",:], description.loc["lb",:])
    description.loc["chosen_upper"] = np.where(description.loc["ub",:] < description.loc["max",:], description.loc["ub",:], description.loc["max",:])

    # visualizing the boundaries
    colour = ['b', 'orange', 'green']
    cluster = [0,1,2]
    for i in range(0,3):
        plt.axvline(x =  description.loc["chosen_lower",cluster[i]], color = colour[i], linestyle = ':')
        plt.axvline(x =  description.loc["chosen_upper",cluster[i]], color = colour[i], linestyle = ':')
        
    # plot the marginal distances
    cluster_sequence = description.loc['chosen_lower',:].T.sort_values().index.to_list()
    
    for i in [0,1]:
        x_value = description.loc["chosen_upper",cluster_sequence[i]]-(description.loc["chosen_upper",cluster_sequence[i]]-description.loc["chosen_lower", cluster_sequence[i+1]])/2
        plt.axvline(x = x_value, color = 'r', label = f'MoE_{cluster_sequence[i],cluster_sequence[i+1]} ≈ {round(x_value,2)}')
        
    plt.legend()
    # plt.savefig(f'../plots/{x_variable}.jpg')
    # plt.clf()
    
    return